In [1]:
import pandas as pd
import numpy as np

# BP Statistical Review
- <b>Goal</b>: Compile coal and gas reserve to production data from the BP Statistical Review of World Energy 2022 dataset for further regression analysis
    - gas reserve, gas production, and coal production have temporal data
    - coal reserve only has data at the end of 2020 -> estimate 
    - we also calculate..."R/P ratios represent the length of time that those remaining reserves would last if production were to continue at the previous year’s rate" (BP report)
- This BP dataset includes 39 countries and regions during 1980-2020. After combining, it has 83 countries and covers the same timeframe. 
- <b>Assumptions</b>
    - !!! so far we don't include data in regions labeled "other..."
    - !!! we use both bituminous and lignite coal for coal reserve from the 2013-2021 BP past report pdf (including R/P values and reserves), which correponds to 2012-2020 values. earlier data are based on the World Energy Council's World Resources Survey 2001, 2004, 2010 pdfs (83 countries, 1999, 2002, 2008 data), while filling the data gaps
    - !!! coal reserve to production is from the BP reports or calculations, whichever is larger
    - !!! assume 2021 data is the same as 2020 (BP's assumption)
- <b>Data citation</b>
    - https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html
    - rest are in the _extract_coal_reserve.xlsx file

### temporal data from BP Statistical Review

In [ ]:
BP_dir = "../../data/global_BP_StatisticalReview/"

In [2]:
fpath = BP_dir+"bp-stats-review-2022-all-data.xlsx"
# gas set up
sheet_names = ["Gas - Proved reserves history ","Gas Production - Bcm","Coal Production - Tonnes"]
unit_list = ["Trillion cubic metres","Billion cubic metres","Million tonnes "]
var_names = ["BP_GasReserve_tM3","BP_GasProduction_bM3","BP_CoalProduction_mTon"]
# coal set up
#coal_sheet_names = []

In [3]:
# compile the datasets that included temporal data
for sheet_select in sheet_names:
    unit_select = unit_list[sheet_names.index(sheet_select)]
    var_select = var_names[sheet_names.index(sheet_select)]
    data_select = pd.read_excel(fpath, sheet_name=sheet_select, skiprows=2)
    data_select = data_select.rename(columns={unit_select:"country"}).set_index("country")
    data_select = data_select.dropna(axis = 0, how = 'all')
    data_select = data_select.rename(index={"                 of which: OECD":"OECD","                 Non-OECD":"Non-OECD",
                                           "                 European Union":"European Union"})
    data_select = data_select.iloc[:,:-3]
    data_select = data_select.reset_index()
    data_select_cols = data_select.columns
    data_select_melt = pd.melt(data_select,id_vars="country", value_vars=data_select_cols,
                               var_name='year',value_name = var_select)
    data_select_melt = data_select_melt.set_index(["country","year"])
    data_select_melt = data_select_melt.dropna()
    if sheet_names.index(sheet_select)==0:
        data_compile = data_select_melt
    else:
        data_compile = pd.concat([data_compile,data_select_melt],axis=1)
data_compile

,,BP_GasReserve_tM3,BP_GasProduction_bM3,BP_CoalProduction_mTon
country,year,,,
Canada,1980,2.429700,71.227950,NaN
Mexico,1980,1.781325,25.115242,NaN
US,1980,5.396133,525.090747,NaN
Total North America,1980,9.607158,621.433939,NaN
Argentina,1980,0.624975,8.164624,NaN
...,...,...,...,...
Zimbabwe,2021,NaN,NaN,3.239215
Japan,2021,NaN,NaN,0.674713
Mongolia,2021,NaN,NaN,32.317600


### filling out non-temporal data (coal reserve)

In [4]:
coal_reserve = pd.read_excel(BP_dir+"_extract_coal_reserve.xlsx",sheet_name="total_reserve")
coal_reserve

,country_wer,country_bp,1999,2002,2008,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Afghanistan,NaN,66,66.0,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,NaN,0,794.0,794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,NaN,40,40.0,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Argentina,NaN,430,424.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Australia,Australia,82 090,78500.0,76 400,76400.0,76400.0,76400.0,76400.0,144818.0,144818.0,147435.0,149079.0,150227.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Uzbekistan,Uzbekistan,4 000,4000.0,1 900,NaN,NaN,1900.0,1900.0,1375.0,1375.0,1375.0,1375.0,1375.0
73,Venezuela,Venezuela,479,479.0,479,479.0,479.0,479.0,479.0,731.0,731.0,731.0,731.0,731.0
74,Vietnam,Vietnam,150,150.0,150,150.0,150.0,150.0,150.0,3360.0,3360.0,3360.0,3360.0,3360.0
75,Zambia,NaN,10,10.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# convert values stored in string to int or float
# because some string valuec contain " " space
def convert_data(value):
    if type(value) is np.float64 or type(value) is int or type(value) is float:
        value_out = value
    elif type(value) is str:
        value_out = int(value.replace(" ",""))
        
    return value_out

In [6]:
for country_name in coal_reserve["country_wer"].unique():
    data_select = coal_reserve.query("country_wer==@country_name")
    data_ind = data_select.index
    # generally assume 
    # 1999, 2002 and 2008 data fill the gap
    if data_select["country_bp"].isna().iloc[0]:
        # for countries only in the WER survey, assume the existing data fill the gap, 
        # latest data apply until 2020
        if data_select.loc[:,1999].isna().iloc[0]:
            data_select.loc[:,np.arange(14)+2008] = convert_data(data_select.loc[data_ind,2008].iloc[0])
        elif data_select.loc[:,2008].isna().iloc[0]:
            data_select.loc[:,np.arange(3)+1999] = convert_data(data_select.loc[data_ind,1999].iloc[0])
            data_select.loc[:,np.arange(19)+2002] = convert_data(data_select.loc[data_ind,2002].iloc[0])
        else:
            data_select.loc[:,np.arange(3)+1999] = convert_data(data_select.loc[data_ind,1999].iloc[0])
            data_select.loc[:,np.arange(6)+2002] = convert_data(data_select.loc[data_ind,2002].iloc[0])
            data_select.loc[:,np.arange(14)+2008] = convert_data(data_select.loc[data_ind,2008].iloc[0])
        data_select = data_select.rename(columns={"country_wer":"country"})

    elif data_select["country_bp"].isna().iloc[0]==False:
        # for countries in the BP survey, assume data gaps to be filled by most recent existing data
        if data_select.loc[:,2012].isna().iloc[0]:
            data_select.loc[:,np.arange(3)+2012] = data_select.loc[data_ind,2015].iloc[0]
        elif data_select.loc[:,2015].isna().iloc[0]:
            data_select.loc[:,np.arange(6)+2015] = data_select.loc[data_ind,2014].iloc[0]
        data_select.loc[:,np.arange(3)+1999] = convert_data(data_select.loc[data_ind,1999].iloc[0])
        data_select.loc[:,np.arange(6)+2002] = convert_data(data_select.loc[data_ind,2002].iloc[0])
        data_select.loc[:,np.arange(4)+2008] = convert_data(data_select.loc[data_ind,2008].iloc[0])
        data_select = data_select.rename(columns={"country_bp":"country"})
    data_select_melt = pd.melt(data_select, id_vars=["country"], value_vars = data_select.columns[2:], var_name = "year", value_name = "BP_CoalReserve_mTon")
    data_select_melt = data_select_melt.sort_values(by="year")
    if data_ind == 0:
        data_coal_reserve = data_select_melt
    else:
        data_coal_reserve = pd.concat([data_coal_reserve,data_select_melt])
data_coal_reserve["BP_CoalReserve_mTon"] = pd.to_numeric(data_coal_reserve["BP_CoalReserve_mTon"], errors='coerce')
data_coal_reserve = data_coal_reserve.set_index(["country","year"])
data_coal_reserve

BP_CoalReserve_mTon
country     year                     
Afghanistan 1999                 66.0
            2000                 66.0
            2001                 66.0
            2002                 66.0
            2003                 66.0
...                               ...
Zimbabwe    2016                502.0
            2017                502.0
            2018                502.0
            2019                502.0
            2020                502.0

[1697 rows x 1 columns]

In [7]:
# add this data to the full dataset
data_compile_coalreserve = pd.concat([data_compile,data_coal_reserve],axis=1)
data_compile_coalreserve

BP_GasReserve_tM3  BP_GasProduction_bM3  \
country             year                                            
Canada              1980           2.429700             71.227950   
Mexico              1980           1.781325             25.115242   
US                  1980           5.396133            525.090747   
Total North America 1980           9.607158            621.433939   
Argentina           1980           0.624975              8.164624   
...                                     ...                   ...   
Zambia              2017                NaN                   NaN   
                    2018                NaN                   NaN   
                    2019                NaN                   NaN   
                    2020                NaN                   NaN   
                    2021                NaN                   NaN   

                          BP_CoalProduction_mTon  BP_CoalReserve_mTon  
country             year                                               
Canada              1980                     NaN                  NaN  
Mexico              1980                     NaN                  NaN  
US                  1980                     NaN                  NaN  
Total North America 1980                     NaN                  NaN  
Argentina           1980                     NaN                  NaN  
...                                          ...                  ...  
Zambia              2017                     NaN                 10.0  
                    2018                     NaN                 10.0  
                    2019                     NaN                 10.0  
                    2020                     NaN                 10.0  
                    2021                     NaN                 10.0  

[4924 rows x 4 columns]

### filling out non-temporal data (coal reserve to production)

In [8]:
# the star means over 500
coal_r2p = pd.read_excel(BP_dir+"_extract_coal_reserve.xlsx",sheet_name="r2p")
coal_r2p.head()

,country,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Australia,177,160,155,158,294,301,304,294,315
1,Brazil,*,*,*,*,*,*,*,*,*
2,Bulgaria,72,83,76,66,75,69,78,153,192
3,Canada,98,95,96,108,109,111,121,130,166
4,China,31,31,30,31,72,39,38,37,37


In [9]:
for country_select in coal_r2p["country"].unique():
    coal_r2p_select = coal_r2p[coal_r2p["country"]==country_select]
    coal_r2p_melt = pd.melt(coal_r2p_select, id_vars=["country"], value_vars = coal_r2p_select.columns[1:], var_name = "year", value_name = "BP_CoalR2P_yr_orig")
    if country_select == coal_r2p["country"].unique()[0]:
        coal_r2p_orig = coal_r2p_melt
    else:
        coal_r2p_orig = pd.concat([coal_r2p_orig,coal_r2p_melt])
coal_r2p_orig = coal_r2p_orig.set_index(["country","year"])
coal_r2p_orig = coal_r2p_orig.replace("*",500)
coal_r2p_orig["BP_CoalR2P_yr_orig"] = pd.to_numeric(coal_r2p_orig["BP_CoalR2P_yr_orig"], errors='coerce')
coal_r2p_orig

BP_CoalR2P_yr_orig
country   year                    
Australia 2012               177.0
          2013               160.0
          2014               155.0
          2015               158.0
          2016               294.0
...                            ...
Zimbabwe  2016               186.0
          2017               171.0
          2018               165.0
          2019               215.0
          2020               153.0

[351 rows x 1 columns]

In [11]:
data_compile_coalreserve["BP_GasR2P_yr"] = data_compile_coalreserve["BP_GasReserve_tM3"]*1000/data_compile_coalreserve["BP_GasProduction_bM3"]
data_compile_coalreserve["BP_CoalR2P_yr_calc"] = data_compile_coalreserve["BP_CoalReserve_mTon"]/data_compile_coalreserve["BP_CoalProduction_mTon"]
data_compile_complete = pd.concat([data_compile_coalreserve,coal_r2p_orig],axis=1)
data_compile_complete["BP_CoalR2P_yr"] = data_compile_complete[["BP_CoalR2P_yr_calc","BP_CoalR2P_yr_orig"]].max(axis=1)
data_compile_complete = data_compile_complete.drop(columns=["BP_CoalR2P_yr_calc","BP_CoalR2P_yr_orig"])
data_compile_complete = data_compile_complete.replace(np.inf, np.nan)

#add_year_data = data_compile_complete.reset_index()
#add_year_data = add_year_data[add_year_data["year"]==2020]
#add_year_data["year"]=2021
#add_year_data = add_year_data.set_index(["country","year"])
#data_compile_2 = pd.concat([data_compile_complete,add_year_data])

data_compile_complete.to_csv("_all_temporal_BPstats.csv")
data_compile_complete

BP_GasReserve_tM3  BP_GasProduction_bM3  \
country             year                                            
Canada              1980           2.429700             71.227950   
Mexico              1980           1.781325             25.115242   
US                  1980           5.396133            525.090747   
Total North America 1980           9.607158            621.433939   
Argentina           1980           0.624975              8.164624   
...                                     ...                   ...   
Middle East         2016                NaN                   NaN   
                    2017                NaN                   NaN   
                    2018                NaN                   NaN   
                    2019                NaN                   NaN   
                    2020                NaN                   NaN   

                          BP_CoalProduction_mTon  BP_CoalReserve_mTon  \
country             year                                                
Canada              1980                     NaN                  NaN   
Mexico              1980                     NaN                  NaN   
US                  1980                     NaN                  NaN   
Total North America 1980                     NaN                  NaN   
Argentina           1980                     NaN                  NaN   
...                                          ...                  ...   
Middle East         2016                     NaN                  NaN   
                    2017                     NaN                  NaN   
                    2018                     NaN                  NaN   
                    2019                     NaN                  NaN   
                    2020                     NaN                  NaN   

                          BP_GasR2P_yr  BP_CoalR2P_yr  
country             year                               
Canada              1980     34.111609            NaN  
Mexico              1980     70.926055            NaN  
US                  1980     10.276572            NaN  
Total North America 1980     15.459661            NaN  
Argentina           1980     76.546699            NaN  
...                                ...            ...  
Middle East         2016           NaN          500.0  
                    2017           NaN          500.0  
                    2018           NaN          500.0  
                    2019           NaN          500.0  
                    2020           NaN          500.0  

[4951 rows x 6 columns]